In [69]:
using DataFrames
using CSV
using Plots
using TextAnalysis
using Dates

In [70]:
ENV["COLUMNS"] = 1000;
ENV["LINES"] = 300;

In [71]:
# load data
df = CSV.read("p1361761_raw.csv", DataFrame);

In [72]:

rem_special_chars(str::String) = replace(str, " " => "_", "/" => "", "@" => "", "*" => "", "." => "_")


nams = StringDocument.(names(df)) 
prepare!.(nams, strip_punctuation)


names_text = TextAnalysis.text.(nams)
names_text = rem_special_chars.(names_text)

rename!(df, names(df) .=> names_text); #rename column names

In [73]:
df |> names

57-element Vector{String}:
 "Document_Date"
 "Object_type"
 "Object"
 "Cost_Element"
 "Document_Number"
 "ValCOArea_Crcy"
 "CO_area_currency"
 "Value_TranCurr"
 "Transaction_Currency"
 "Purchasing_Document"
 "Purchase_order_text"
 "Project_Definition"
 "WBS_Element"
 "Partner_object"
 "Partner_object_type"
 "Posting_Date"
 "Created_on"
 "User_Name"
 "Reversal_document"
 "Refdocument_type"
 "Ref_Document_Number"
 "Document_type"
 "Value_Type"
 "Name"
 "Trading_Partner"
 "Functional_Area"
 "CO_object_name"
 "Object_type1"
 "Partner_object_type1"
 "Cost_element_name"
 "Cost_element_descr"
 "Offsetting_acct_no"
 "Offsetting_account_type"
 "Total_Quantity"
 "Total_quantity"
 "Personnel_Number"
 "Name_of_employee_or_applicant"
 "Description"
 "Business_Transaction"
 "Document_Header_Text"
 "Ref_company_code"
 "Original_bus_trans"
 "Item"
 "Business_Area"
 "Company_Code"
 "Partner_Object_Class"
 "FI_Posting_Item"
 "CO_partner_object_name"
 "PartnerCCtr"
 "Source_Object"
 "Source_object_name"


In [74]:
## generic groupby, combine, apply
# grpCombApply(df, grp::String, comb::String, f::Function)
#		Args:
#			`df:: dataframe`
#			`grp:: String`: Column name to group by
# 			`comb:: String`: Column name to combine
#			`f:: Function`: Function to apply to the combined column
function grpCombApply(df, grp::String, comb::String, f::Function)
    dfg = groupby(df, grp);
    combine(dfg, comb => f);
end

grpCombApply (generic function with 1 method)

In [75]:
#groupby columns "CO_object_name" and "CO_partner_object_name"

dfg = groupby(df, [:CO_object_name, :CO_partner_object_name, :Source_object_name])

#combine sum of :Total_Quantity
dfg_sum = combine(dfg, :Total_Quantity => sum)

Row,CO_object_name,CO_partner_object_name,Source_object_name,Total_Quantity_sum
,String,String?,String31?,Float64
1,Controls PS Engineering Labor General,CS - Tech Supp Desk / Labor DI,CS - Tech Supp Desk / Labor DI,12.0
2,PS Project Warranty,missing,missing,0.0
3,Integrated (WCS),CS - Tech Supp Desk / Labor DI,CS - Tech Supp Desk / Labor DI,7.0
4,Mechanical Installation Labor PERM,CS - Tech Supp Desk / Labor DI,CS - Tech Supp Desk / Labor DI,31.45
5,PS Special Cost,missing,missing,0.0
6,Mechanical Engineering Labor General,CS-Parts Spares Supp / Labor DI,CS-Parts Spares Supp / Labor DI,1.0
7,Third Party Material - PS Mechanical,REPAIR:50mm PWR RLR 388.7mm SHL 0-GRV SI,missing,0.0
8,Third Party Material - PS Mechanical,50MM CPLD PWR RLR 388.7MM SHL 0-GRV SGL,missing,0.0
9,PS Project,missing,missing,0.0


In [76]:
CSV.write("dfg_sum$(Dates.today()).csv", dfg_sum);

In [81]:
c = 24024.00 + 60816.00 + 18765

103605.0

In [82]:
c_per_ft_conv = c / 5718.00

18.11909758656873

In [83]:
new_c = c_per_ft_conv * 6939.0

125728.41815320043

In [84]:
# take 20% off the above cost

new_c * 0.8

100582.73452256035

In [85]:
#convert above to effort
rate = 84.0
effort = new_c / rate

1496.7668827761956

In [ ]:
#convert above to effort
rate = 84.0
effort = new_c / rate

1496.7668827761956

In [ ]:
#convert above to effort
rate = 84.0
effort = new_c / rate

1496.7668827761956